In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from ast import literal_eval

In [2]:
df = pd.read_csv('movies.csv')

# Selecciona las columnas que queremos del df
df = df[['title', 'vote_average','overview','vote_count','popularity']]

# Guardamos el nuevo dataset en un archivo CSV
df.to_csv('moviesml.csv', index=False)

In [3]:
df = pd.read_csv('moviesml.csv')
df.head()

,title,vote_average,overview,vote_count,popularity
0,Toy Story,7.7,"Led by Woody, Andy's toys live happily in his ...",5415.0,21.946943
1,Jumanji,6.9,When siblings Judy and Peter discover an encha...,2413.0,17.015539
2,Grumpier Old Men,6.5,A family wedding reignites the ancient feud be...,92.0,11.712900
3,Waiting to Exhale,6.1,"Cheated on, mistreated and stepped on, the wom...",34.0,3.859495
4,Father of the Bride Part II,5.7,Just when George Banks has recovered from his ...,173.0,8.387519


###### Cálculo de valores estadísticos: Se calculan estadísticas sobre los campos 'vote_count' y 'vote_average'. Se obtiene la media de los valores de 'vote_average' y se almacena en la variableC . Además, se calcula el percentil 0,95 de los valores de 'vote_count' y se almacena en la variable m

In [4]:
vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

5.250176304654443

In [5]:
m = vote_counts.quantile(0.95)
m

434.0

###### Filtrado de películas calificadas: Se seleccionan las películas que cumplen ciertos criterios de calificación, como tener un número mínimo de votos ('vote_count' mayor o igual am y tener calificaciones ('vote_average' no nulas). Las columnas 'title', 'vote_count', 'vote_average' y 'popularity' de las películas calificadas se asignan al DataFrame qualified.

In [6]:
qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'vote_count', 'vote_average', 'overview','popularity']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape
qualified.head()

,title,vote_count,vote_average,overview,popularity
0,Toy Story,5415,7,"Led by Woody, Andy's toys live happily in his ...",21.946943
1,Jumanji,2413,6,When siblings Judy and Peter discover an encha...,17.015539
5,Heat,1886,7,"Obsessive master thief, Neil McCauley leads a ...",17.924927
9,GoldenEye,1194,6,James Bond must unmask the mysterious head of ...,14.686036
15,Casino,1343,7,The life of the gambling paradise – Las Vegas ...,10.137389


##### Cálculo de la puntuación ponderada: Se define la función weighted_rating(x)que calcula la puntuación ponderada para una película dada, utilizando la fórmula específica modificada en el código. Esta función se aplica al DataFramequalified utilizando el método apply(), y los resultados se agregan como una nueva columna llamada 'wr' en el DataFrame qualified.

In [7]:

def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [8]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [9]:
qualified.head(15) #las mejores peliculas

,title,vote_count,vote_average,overview,popularity,wr
0,Toy Story,5415,7,"Led by Woody, Andy's toys live happily in his ...",21.946943,6.870162
1,Jumanji,2413,6,When siblings Judy and Peter discover an encha...,17.015539,5.885696
5,Heat,1886,7,"Obsessive master thief, Neil McCauley leads a ...",17.924927,6.672662
9,GoldenEye,1194,6,James Bond must unmask the mysterious head of ...,14.686036,5.800108
15,Casino,1343,7,The life of the gambling paradise – Las Vegas ...,10.137389,6.572637
17,Four Rooms,539,6,It's Ted the Bellhop's first night on the job....,9.026586,5.665546
18,Ace Ventura: When Nature Calls,1128,6,"Summoned from an ashram in Tibet, Ace finds hi...",8.205448,5.791662
31,Twelve Monkeys,2470,7,"In the year 2035, convict James Cole reluctant...",12.297305,6.738491
33,Babe,756,6,Babe is a little pig who doesn't quite know hi...,14.404764,5.726535
38,Clueless,828,6,"Shallow, rich and socially successful Cher is ...",9.882383,5.742137


##### Creación de la matriz TF-IDF: Se utiliza la clase TfidfVectorizerde scikit-learn para crear una matriz de término-frecuencia de documento-frecuencia inversa (TF-IDF) a partir de los textos de resumen ('overview') de las películas en el marco de datos qualified. La matriz TF-IDF se asigna a la variable tfidf_matrix

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(qualified['overview'].fillna(''))
print(tfidf_matrix.shape)

(2274, 68840)


##### Cálculo de la similitud del coseno: Se utiliza la función linear_kernel()de scikit-learn para calcular la similitud del coseno entre las filas de la matriz TF-IDF tfidf_matrix. El resultado se asigna a la variable cosine_sim.

In [8]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

##### Configuración de índices y títulos: Se redefine el índice del DataFrame dfutilizando el método reset_index(). Además, se crea una Serie de pandas indicesque mapea los títulos de las películas llamadas a sus índices correspondientes en el DataFrame.

In [9]:
df = df.reset_index()
titles =df['title']
indices = pd.Series(df.index, index=df['title'])

##### Definición de la función de recomendación: Se define la función recomendacion(titulo)que toma un título de película como argumento y utiliza la matriz de similitud del coseno cosine_sim, los índicesindices y los títulostitles para recomendar las 5 películas más. La función devuelve una lista de los títulos de las películas recomendadas.

In [10]:
def recomendacion(titulo):
    # Convertimos el título ingresado a minúsculas
    titulo = titulo.lower()
    
    # Creamos un nuevo diccionario con los títulos en minúsculas
    indices_lower = {k.lower(): v for k, v in indices.items()}
    
    # Obtenemos el índice de la película ingresada utilizando el diccionario con los títulos en minúsculas
    idx = indices_lower[titulo]
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # Obtener los 5 puntajes de similitud más altos
    movie_indices = [i[0] for i in sim_scores]
    top_movies = titles.iloc[movie_indices].tolist()  # Convertir los títulos en una lista de Python
    return top_movies

In [11]:
recomendacion("Casino")

['M. Butterfly', 'Friday', 'Antz', 'The Pest', "L'Enfer"]